In [ ]:
import numpy as np

import efd_utils
import df_utils

In [ ]:
location = "summit"

In [ ]:
client = efd_utils.get_client(location)
cscs = efd_utils.get_cscs(f"{location}_systems.dat")

In [ ]:
# Full shutdown goes to OFFLINE state. Normal is to STANDBY state.
do_full_shutdown = False
if do_full_shutdown:
    ss_limit = 4
    shutdown_start = 0
else:
    ss_limit = 3
    shutdown_start = 1
ss_shutdown_order = np.array([4, 5, 1, 2])
# Give CSCs ten minutes to shutdown
shutdown_wait_time = 600  # seconds

In [ ]:
print("######################################")
print("#        Shutdown Procedure          #")
print("######################################")
for csc in cscs:
    query = efd_utils.get_base_query(columns=["private_sndStamp",
                                              "summaryState"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name="logevent_summaryState")
    # Handle indexed components from base query
    if csc.index != 0 and "WHERE" not in query:
        query += " WHERE"
    query += " " + efd_utils.get_time_clause(last=True, limit=ss_limit)
    #print(query)
    ss_df = await client.query(query)
    ss_df = df_utils.convert_timestamps(ss_df, ["private_sndStamp"])

    print("--------------------------------------")
    print(f"CSC: {csc.full_name}")

    ss_shutdown = ss_df.summaryState.values
    does_shutdown = np.all(ss_shutdown == ss_shutdown_order[shutdown_start:])
    if not does_shutdown:
        print(f"Incorrect Shutdown Order: {ss_shutdown}")
    else:
        print("Shutdown Order Correct!")
        shutdown_time = df_utils.time_delta(ss_df.private_sndStamp.values[0],
                                            ss_df.private_sndStamp.values[-1])
        print(f"Total Shutdown Time: {shutdown_time:.2f} s")
        if shutdown_time / np.timedelta64(1, 'ns') > shutdown_wait_time:
            print("Timestamps:")
            for timestamp in ss_df.private_sndStamp.values:
                print(f"\t{timestamp}")